In [1]:
import tensorflow as tf

from data_loader.kitty_road_data_loader import KittyRoadLoader
from models.fcn_alexnet_model import FcnAlexnetModel
from trainers.example_trainer import ExampleTrainer
from utils.config import process_config
from utils.dirs import create_dirs
from utils.logger import Logger
from utils.utils import get_args
import matplotlib.pyplot as plt
import os 
json_file = "configs/fcn_alexnet_KittyRoadDataset_config.json"
json_file

config = process_config(json_file)
print(config)



batch_size: 16
checkpoint_dir: ./experiments\fcn_alexnet_kittyroad\checkpoint/
data_path: C:/Users/TT260256/My Project/NNs/data_road
exp_name: fcn_alexnet_kittyroad
image_size:
- 320
- 480
learning_rate: 0.001
max_to_keep: 5
num_epochs: 10
num_iter_per_epoch: 10
state_size:
- 784
summary_dir: ./experiments\fcn_alexnet_kittyroad\summary/



In [2]:
data = KittyRoadLoader(config)
img, mask = data.get_data_element("all_data",0)

Yeah this is the path to dataset : C:/Users/TT260256/My Project/NNs/data_road
Size of all raw images :  289 samples with size  (375, 1242, 3)
Size of all raw labels  :  289 samples with size  (375, 1242, 3)
Size of all raw masks  :  289 samples with size  (375, 1242)
Pre-processing the data...
Size of images collection :  (2312, 320, 480, 3)
Size of masks collection :  (2312, 320, 480)


In [3]:
# img, mask = data.get_data_element("all_data",2)
# plt.imshow(img)
# plt.show()
# plt.imshow(mask)
# plt.show()

In [4]:
model = FcnAlexnetModel(config)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 320, 480, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 78, 118, 96)       34944     
_________________________________________________________________
activation (Activation)      (None, 78, 118, 96)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 59, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 39, 59, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 39, 59, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 39, 59, 256)       0         
__________

In [9]:
with tf.Session() as sess :
    # create tensorboard logger
    logger = Logger(sess, config)

AttributeError: 'FcnAlexnetModel' object has no attribute 'train_data'

In [ ]:
# with tf.Session() as sess : 
#     writer = tf.summary.FileWriter("/tmp/log/TestTemp", sess.graph)
    
# writer.flush()
# writer.close()